Notebook to prepare features for final modelling

### Libraries used

In [25]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler

### Load data

In [2]:
claims_data = pd.read_csv('../Dataset/FreMTPL2freq.csv')
display(claims_data.head())

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
0,1.0,1,0.10,D,5,0,55,50,B12,Regular,1217,R82
1,3.0,1,0.77,D,5,0,55,50,B12,Regular,1217,R82
2,5.0,1,0.75,B,6,2,52,50,B12,Diesel,54,R22
3,10.0,1,0.09,B,7,0,46,50,B12,Diesel,76,R72
4,11.0,1,0.84,B,7,0,46,50,B12,Diesel,76,R72


In [3]:
claims_data.shape

(678013, 12)

In [9]:
# calculate binary variable Claim, which states whether at least 1 claims has happened for the policy
claims_data['Claim'] = claims_data['ClaimNb'].apply(lambda x: 1 if x >=1 else 0)

### Final data checks

In [10]:
# check for exposures > 1
claims_data[claims_data['Exposure']>1]

,IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region,Claim
179652,1179653.0,0,1.99,A,11,10,50,50,B12,Regular,29,R25,0
179653,1179654.0,1,1.17,D,4,2,21,112,B6,Regular,721,R93,1
179655,1179656.0,1,1.12,E,6,0,37,72,B4,Diesel,9307,R82,1
179657,1179658.0,0,1.48,D,7,4,46,50,B6,Diesel,776,R93,0
179659,1179660.0,1,1.50,C,8,14,26,76,B1,Regular,372,R25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349346,2282494.0,0,1.06,A,5,14,62,50,B3,Regular,43,R82,0
352404,2285552.0,0,1.02,E,7,13,42,60,B11,Regular,3317,R93,0
352454,2285602.0,0,1.01,E,4,16,59,51,B1,Diesel,6736,R11,0
353728,2286877.0,0,1.18,C,9,16,25,68,B12,Regular,462,R72,0


Conclusion:
- It is strange that exposure is > 1 for some records -> in reality I would clarifiy this with the data owner
- for the current exercise records are taken into account as their overall portion is rather small and deletion seems unnecessary

In [11]:
# Check nr. of claims
claims_data['ClaimNb'].value_counts()

ClaimNb
0     643953
1      32178
2       1784
3         82
4          7
11         3
5          2
6          1
8          1
16         1
9          1
Name: count, dtype: int64

Conclusion:
- It 4+ claims in a year per policy seems strange, but again the number is low and should not be disturbing too much -> no records cleaned

### Applying train-test split

In [14]:
# train-test split should be applied before the feature transformation
X_train, X_test, y_train, y_test = train_test_split(claims_data.drop(['IDpol', 'ClaimNb', 'Claim'], axis = 1), claims_data[['Claim']] , test_size=0.2, random_state=42)
y_train.head()

,Claim
406477,1
393150,0
252885,0
260652,0
661256,0


### Ordinal Encoding of Area

In [27]:
X_train.head()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
406477,1.00,B,6,5,59,50,B2,Diesel,96,R72
393150,1.00,C,10,11,61,50,B3,Regular,111,R91
252885,1.00,A,7,4,42,50,B1,Diesel,19,R72
260652,1.00,A,6,4,52,50,B2,Diesel,16,R82
661256,0.28,D,5,3,71,50,B12,Regular,538,R53


In [58]:
# do ordinal encoding of the area variable
ord_enc = OrdinalEncoder()
ord_area = ord_enc.fit_transform(X_train['Area'].values.reshape(-1,1))
ord_area[:5].ravel()

array([1., 2., 0., 0., 3.])

In [61]:
# overwrite in training data
X_train['Area'] = ord_area.ravel()
X_train.head()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
406477,1.00,1.0,6,5,59,50,B2,Diesel,96,R72
393150,1.00,2.0,10,11,61,50,B3,Regular,111,R91
252885,1.00,0.0,7,4,42,50,B1,Diesel,19,R72
260652,1.00,0.0,6,4,52,50,B2,Diesel,16,R82
661256,0.28,3.0,5,3,71,50,B12,Regular,538,R53


In [62]:
# apply to training data
ord_area = ord_enc.fit_transform(X_test['Area'].values.reshape(-1,1))
X_test['Area'] = ord_area.ravel()
X_test.head()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
261354,0.43,3.0,7,18,36,95,B1,Regular,1054,R24
448143,0.10,3.0,7,17,80,95,B2,Regular,598,R25
188618,0.33,4.0,7,3,36,76,B6,Regular,4172,R82
12952,0.56,0.0,5,4,73,52,B13,Diesel,15,R24
425028,0.27,4.0,8,0,37,50,B11,Diesel,3021,R53


### Treating numerical features

on the numerical variables two types of sacling are applied:
- MinMax Scaler
- StandardScaler

In [64]:
X_train.head()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
406477,1.00,1.0,6,5,59,50,B2,Diesel,96,R72
393150,1.00,2.0,10,11,61,50,B3,Regular,111,R91
252885,1.00,0.0,7,4,42,50,B1,Diesel,19,R72
260652,1.00,0.0,6,4,52,50,B2,Diesel,16,R82
661256,0.28,3.0,5,3,71,50,B12,Regular,538,R53


In [65]:
# get numerical columns
numeric_cols = X_train.select_dtypes(include='number').columns
numeric_cols

Index(['Exposure', 'Area', 'VehPower', 'VehAge', 'DrivAge', 'BonusMalus',
       'Density'],
      dtype='object')

In [66]:
# 1st: scale according to MinMax Scaler
scaler_mm = MinMaxScaler()
X_train_sc_mm = scaler_mm.fit_transform(X_train[numeric_cols])
X_train_sc_mm = pd.DataFrame(X_train_sc_mm, columns = numeric_cols)
X_train_sc_mm.describe()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,Density
count,542410.000000,542410.000000,542410.000000,542410.000000,542410.000000,542410.000000,542410.000000
mean,0.262126,0.457755,0.223031,0.070422,0.335341,0.054229,0.066303
std,0.181548,0.276508,0.186212,0.056565,0.172406,0.086906,0.146766
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.088313,0.200000,0.090909,0.020000,0.195122,0.000000,0.003370
50%,0.242752,0.400000,0.181818,0.060000,0.317073,0.000000,0.014482
75%,0.496828,0.600000,0.272727,0.110000,0.451220,0.077778,0.060336
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [67]:
# 2nd: scale according to StandardScaler
scaler_st = StandardScaler()
X_train_sc_st = scaler_st.fit_transform(X_train[numeric_cols])
X_train_sc_st = pd.DataFrame(X_train_sc_st, columns = numeric_cols)
X_train_sc_st.describe()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,Density
count,5.424100e+05,5.424100e+05,5.424100e+05,5.424100e+05,5.424100e+05,5.424100e+05,5.424100e+05
mean,6.257416e-17,-1.629083e-16,1.867498e-16,4.467010e-18,1.085510e-16,7.526453e-17,3.411171e-17
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.443838e+00,-1.655487e+00,-1.197724e+00,-1.244960e+00,-1.945066e+00,-6.239956e-01,-4.517610e-01
25%,-9.573944e-01,-9.321804e-01,-7.095224e-01,-8.913864e-01,-8.133081e-01,-6.239956e-01,-4.287958e-01
50%,-1.067177e-01,-2.088736e-01,-2.213204e-01,-1.842397e-01,-1.059591e-01,-6.239956e-01,-3.530865e-01
75%,1.292783e+00,5.144331e-01,2.668815e-01,6.996937e-01,6.721247e-01,2.709688e-01,-4.065914e-02
max,4.064342e+00,1.961047e+00,4.172497e+00,1.643371e+01,3.855195e+00,1.088269e+01,6.361830e+00


In [68]:
# apply to X_test
X_test_sc_mm = pd.DataFrame(scaler_mm.transform(X_test[numeric_cols]), columns = numeric_cols)
X_test_sc_st = pd.DataFrame(scaler_st.transform(X_test[numeric_cols]), columns = numeric_cols)
display(X_test_sc_mm.describe())
display(X_test_sc_st.describe())

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,Density
count,135603.000000,135603.000000,135603.000000,135603.000000,135603.000000,135603.000000,135603.000000
mean,0.261779,0.458677,0.223619,0.070527,0.335411,0.054237,0.066545
std,0.181612,0.276652,0.187378,0.057048,0.172415,0.086727,0.146046
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000037
25%,0.083331,0.200000,0.090909,0.020000,0.195122,0.000000,0.003408
50%,0.242752,0.400000,0.181818,0.060000,0.317073,0.000000,0.014556
75%,0.491847,0.600000,0.272727,0.110000,0.451220,0.083333,0.061521
max,1.000000,1.000000,1.000000,1.000000,0.987805,0.988889,1.000000


,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,Density
count,135603.000000,135603.000000,135603.000000,135603.000000,135603.000000,135603.000000,135603.000000
mean,-0.001914,0.003334,0.003157,0.001862,0.000405,0.000092,0.001653
std,1.000352,1.000521,1.006261,1.008541,1.000053,0.997945,0.995100
min,-1.443838,-1.655487,-1.197724,-1.244960,-1.945066,-0.623996,-0.451509
25%,-0.984836,-0.932180,-0.709522,-0.891386,-0.813308,-0.623996,-0.428543
50%,-0.106718,-0.208874,-0.221320,-0.184240,-0.105959,-0.623996,-0.352582
75%,1.265341,0.514433,0.266882,0.699694,0.672125,0.334895,-0.032583
max,4.064342,1.961047,4.172497,16.433708,3.784460,10.754837,6.361830


### One-hot encoding of categorical variables

In [69]:
# get categorical columns
cat_cols = X_train.select_dtypes(include='object').columns
cat_cols

Index(['VehBrand', 'VehGas', 'Region'], dtype='object')

In [70]:
# create encoder and fit to train data
encoder = OneHotEncoder(categories='auto', sparse=False, handle_unknown='error')
X_train_enc = encoder.fit_transform(X_train[cat_cols])
X_train_enc = pd.DataFrame(X_train_enc, columns = encoder.get_feature_names_out())
X_train_enc

C:\Users\Lenovo\miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,VehBrand_B1,VehBrand_B10,VehBrand_B11,VehBrand_B12,VehBrand_B13,VehBrand_B14,VehBrand_B2,VehBrand_B3,VehBrand_B4,VehBrand_B5,...,Region_R53,Region_R54,Region_R72,Region_R73,Region_R74,Region_R82,Region_R83,Region_R91,Region_R93,Region_R94
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542405,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
542406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
542407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
542408,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
# check categories
encoder.categories_

[array(['B1', 'B10', 'B11', 'B12', 'B13', 'B14', 'B2', 'B3', 'B4', 'B5',
        'B6'], dtype=object),
 array(['Diesel', 'Regular'], dtype=object),
 array(['R11', 'R21', 'R22', 'R23', 'R24', 'R25', 'R26', 'R31', 'R41',
        'R42', 'R43', 'R52', 'R53', 'R54', 'R72', 'R73', 'R74', 'R82',
        'R83', 'R91', 'R93', 'R94'], dtype=object)]

In [72]:
# apply to test data
X_test_enc = encoder.fit_transform(X_test[cat_cols])
X_test_enc = pd.DataFrame(X_test_enc, columns = encoder.get_feature_names_out())
X_test_enc

C:\Users\Lenovo\miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,VehBrand_B1,VehBrand_B10,VehBrand_B11,VehBrand_B12,VehBrand_B13,VehBrand_B14,VehBrand_B2,VehBrand_B3,VehBrand_B4,VehBrand_B5,...,Region_R53,Region_R54,Region_R72,Region_R73,Region_R74,Region_R82,Region_R83,Region_R91,Region_R93,Region_R94
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135598,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135599,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
135600,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
135601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Combine numerical and categorical features into one matrix again

In [73]:
# minmax scaled and one-hot encoded data _ TRAIN
X_train_sc_mm = pd.concat([X_train_sc_mm, X_train_enc], axis = 1)
X_train_sc_mm.head()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,Density,VehBrand_B1,VehBrand_B10,VehBrand_B11,...,Region_R53,Region_R54,Region_R72,Region_R73,Region_R74,Region_R82,Region_R83,Region_R91,Region_R93,Region_R94
0,0.496828,0.2,0.181818,0.05,0.500000,0.0,0.003519,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.496828,0.4,0.545455,0.11,0.524390,0.0,0.004074,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.496828,0.0,0.272727,0.04,0.292683,0.0,0.000667,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.496828,0.0,0.181818,0.04,0.414634,0.0,0.000556,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.138132,0.6,0.090909,0.03,0.646341,0.0,0.019890,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
# Standardscaler scaled and one-hot encoded data _ TRAIN
X_train_sc_st = pd.concat([X_train_sc_st, X_train_enc], axis = 1)
X_train_sc_st.head()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,Density,VehBrand_B1,VehBrand_B10,VehBrand_B11,...,Region_R53,Region_R54,Region_R72,Region_R73,Region_R74,Region_R82,Region_R83,Region_R91,Region_R93,Region_R94
0,1.292783,-0.932180,-0.221320,-0.361026,0.955064,-0.623996,-0.427786,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.292783,-0.208874,1.731487,0.699694,1.096534,-0.623996,-0.424001,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.292783,-1.655487,0.266882,-0.537813,-0.247429,-0.623996,-0.447218,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.292783,-1.655487,-0.221320,-0.537813,0.459920,-0.623996,-0.447976,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-0.682983,0.514433,-0.709522,-0.714600,1.803883,-0.623996,-0.316241,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
# minmax scaled and one-hot encoded data _ TEST
X_test_sc_mm = pd.concat([X_test_sc_mm, X_test_enc], axis = 1)
X_test_sc_mm.head()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,Density,VehBrand_B1,VehBrand_B10,VehBrand_B11,...,Region_R53,Region_R54,Region_R72,Region_R73,Region_R74,Region_R82,Region_R83,Region_R91,Region_R93,Region_R94
0,0.212860,0.6,0.272727,0.18,0.219512,0.250000,0.039001,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.048458,0.6,0.272727,0.17,0.756098,0.250000,0.022112,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.163041,0.8,0.272727,0.03,0.219512,0.144444,0.154487,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.277625,0.0,0.090909,0.04,0.670732,0.011111,0.000519,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.133150,0.8,0.363636,0.00,0.231707,0.000000,0.111856,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
# Standardscaler scaled and one-hot encoded data _ TEST
X_test_sc_st = pd.concat([X_test_sc_st, X_test_enc], axis = 1)
X_test_sc_st.head()

,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,Density,VehBrand_B1,VehBrand_B10,VehBrand_B11,...,Region_R53,Region_R54,Region_R72,Region_R73,Region_R74,Region_R82,Region_R83,Region_R91,Region_R93,Region_R94
0,-0.271365,0.514433,0.266882,1.937200,-0.671838,2.252676,-0.186021,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.176924,0.514433,0.266882,1.760414,2.440497,2.252676,-0.301099,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.545777,1.237740,0.266882,-0.714600,-0.671838,1.038081,0.600852,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.085371,-1.655487,-0.709522,-0.537813,1.945353,-0.496144,-0.448228,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.710424,1.237740,0.755083,-1.244960,-0.601103,-0.623996,0.310380,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Save final transformations into csv-files

In [77]:
# Train data
X_train_sc_mm.to_csv('../Dataset/X_train_sc_mm.csv', index = False)
X_train_sc_st.to_csv('../Dataset/X_train_sc_st.csv', index = False)
y_train.to_csv('../Dataset/y_train.csv', index = False)

In [78]:
# Test data
X_test_sc_mm.to_csv('../Dataset/X_test_sc_mm.csv', index = False)
X_test_sc_st.to_csv('../Dataset/X_test_sc_st.csv', index = False)
y_test.to_csv('../Dataset/y_test.csv', index = False)